## CBA / DMG2 / Supervised Learning Homework Assignment - 1


Submitted By: <b>Amit Kumar Sharma (PGID: 71721079)<b>

### Problem 4 – MNSIT Bayesian

    •	Take the MNIST dataset. Lets call it D0 dataset
    •	Do a 9 dimensional PCA projection. Lets call it D1 dataset
    •	Do a 9 dimensional FISHER projection. Lets call it D2 dataset
    •	Build a Bayesian classifier on D1 (single Gaussian per class)
        •	Diagonal Covariance matrix (i.e.set non-diagonals to zero) 
        •	Full Covariance matrix
    •	Build a Bayesian classifier on D2 (single Gaussian per class)
        •	Diagonal Covariance
        •	Full covariance
    •	Compare the test accuracies of the four classifiers and comment.

In [1]:
import pandas as pd
import os
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import naive_bayes
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

##### Take the MNIST dataset. Lets call it D0 dataset

In [2]:
train = pd.DataFrame()
for i in range(0,10):
    file = '/Users/a5sharma/Documents/ISB/DMG2/Assignment/data/MNIST/train'+str(i)+'.csv'
    temp = pd.read_csv(file,low_memory=False)
    temp = temp.iloc[:,1:]
    temp['label'] = i
    if i == 0:
        train = temp
    else:
        train = pd.concat([train, temp],ignore_index=True)

In [3]:
test = pd.DataFrame()
for i in range(0,10):
    file = '/Users/a5sharma/Documents/ISB/DMG2/Assignment/data/MNIST/test'+str(i)+'.csv'
    temp = pd.read_csv(file,low_memory=False)
    temp = temp.iloc[:,1:]
    temp['label'] = i
    if i == 0:
        test = temp
    else:
        test = pd.concat([test, temp],ignore_index=True)

###### Missing Values
There are lots of missing values in training and testing dataset and all variables are empty for all missing rows. We need to remove all the missing rows further processing.

In [4]:
train.apply(lambda x: sum(x.isnull()))

V1       391
V2       391
V3       391
V4       391
V5       391
V6       391
V7       391
V8       391
V9       391
V10      391
V11      391
V12      391
V13      391
V14      391
V15      391
V16      391
V17      391
V18      391
V19      391
V20      391
V21      391
V22      391
V23      391
V24      391
V25      391
V26      391
V27      391
V28      391
V29      391
V30      391
        ... 
V756     391
V757     391
V758     391
V759     391
V760     391
V761     391
V762     391
V763     391
V764     391
V765     391
V766     391
V767     391
V768     391
V769     391
V770     391
V771     391
V772     391
V773     391
V774     391
V775     391
V776     391
V777     391
V778     391
V779     391
V780     391
V781     391
V782     391
V783     391
V784     391
label      0
Length: 785, dtype: int64

In [5]:
train.dropna(axis=0,inplace=True)
test.dropna(axis=0,inplace=True)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35750 entries, 0 to 36140
Columns: 785 entries, V1 to label
dtypes: float64(784), int64(1)
memory usage: 214.4 MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24250 entries, 0 to 24518
Columns: 785 entries, V1 to label
dtypes: float64(784), int64(1)
memory usage: 145.4 MB


In [8]:
train.iloc[:,0:784].head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V775,V776,V777,V778,V779,V780,V781,V782,V783,V784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


###### Normalization

Lets Normalize both the datasets before applying LDA or PCA.

In [9]:
X_train = train.iloc[:,0:784]/255.0
y_train = train["label"]

X_test = test.iloc[:,0:784]/255.0
y_test = test["label"]

##### Do a 9 dimensional PCA projection. Lets call it D1 dataset

In [10]:
## Set the seed
np.random.seed(1234)

## Generate 9 Principle Components
pca = PCA(n_components = 9)
pca.fit(X_train)
print(pca.explained_variance_)
print('Explained variation per principal component: {0}'.format(pca.explained_variance_ratio_))
print('Explained cumulative variation for 2 principal components: {0}'.format(pca.explained_variance_ratio_.cumsum()))

col_names = [ 'pca'+str(i) for i in range(0,9) ]
digit_pca_9 = pd.DataFrame(pca.transform(X_train), columns = col_names)

Y = pd.DataFrame(np.array(y_train), columns=["label"])

d1 = pd.concat([digit_pca_9, Y], axis=1)

print("The dimensions of PCA projection Data Frame: {0}".format(digit_pca_9.shape))

d1.head(10)

[5.10335982 3.74717042 3.23910925 2.83450361 2.57223319 2.26820366
 1.72876292 1.51358699 1.44555369]
Explained variation per principal component: [0.09694147 0.07117981 0.06152888 0.05384314 0.04886116 0.04308593
 0.03283892 0.02875152 0.02745918]
Explained cumulative variation for 2 principal components: [0.09694147 0.16812128 0.22965016 0.2834933  0.33235446 0.37544039
 0.4082793  0.43703082 0.46449001]
The dimensions of PCA projection Data Frame: (35750, 9)


,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,label
0,4.028728,-1.062822,2.442987,-1.830775,-3.152218,-0.620715,-0.152372,-0.399007,0.213673,0
1,4.122160,-1.303177,2.160442,-2.171781,-3.408331,0.089392,-0.173005,0.207913,1.964713,0
2,2.412930,-0.853290,2.724968,-2.498519,1.573210,-1.656461,0.004307,1.988436,0.456747,0
3,6.532352,-0.329908,-0.559258,-3.094260,-2.990228,1.832665,-0.603947,-1.599748,1.226512,0
4,5.162878,0.045781,-1.127272,-1.032655,-2.616755,2.245742,1.285621,-2.519399,1.953335,0
5,4.828110,-0.635625,-0.698679,-2.246634,1.644010,-1.638070,0.584425,0.952081,0.928532,0
6,3.342471,-1.600990,0.796915,-3.047240,-0.912178,-0.780914,-0.960354,1.927253,1.115426,0
7,4.219502,-0.822439,1.583716,-3.512004,-0.030552,-1.995429,-0.691501,1.047912,-0.413527,0
8,4.595544,-1.395223,-2.461960,0.507267,-2.793008,2.513274,0.632571,-1.514705,0.039801,0
9,5.399449,0.019286,-0.060062,-1.375413,-4.335054,0.696019,1.522625,-1.521426,2.403929,0


##### Do a 9 dimensional FISHER projection. Lets call it D2 dataset

LinearDiscriminantAnalysis is giving "Variables are collinear." warning because there are many variables are linearly related to each other. Also, lots are variables having 0 values for all the observation.

I tried to remove all the variables which is having 0 values and run LDA but still got the same warning. Then, I tried to fit a LinearRegression in R and checked VIF for my LinearRegression model in R. After that, I found V13,V15,V393,V644,V727 variables are linearly related. I removing these variables, "Variables are collinear." warning is gone. But we got same projection before removing the variables and after removing the variables. In this workbook, I have not removed these variables as they is no changes before and after.

    #R
    X = data[,1:ncol(data)-1]
    total = dim(X)[1]
    colName = colnames(X)
    ls = list()
    j = 1
    for (i in 1:length(colName)){
      col = colName[i]
      size = dim(X[X[[i]]==0,])[1]
      print(col)
      if (size == total){
        ls[j] = i
        j = j + 1
      }
    }
    X1 = X[,-c(array((unlist(ls))),13,15,393)]
    X1$label = data$label
    model = lm(label ~ ., data = X1)
    ld.vars <- attributes(alias(model)$Complete)$dimnames[[1]]
    formula.new <- as.formula(
      paste(
        paste(deparse(label ~ .), collapse=""), 
        paste(ld.vars, collapse="-"),
        sep="-"
      )
    )
    data = data[,-c(as.array(unlist(ls)),13,15,393)]
    lda <- lda(label ~ . - V644 - V727 , data = data)

    #Python
    mismatch = False
    for i in range(d2.shape[0]):
        x1 = d2.iloc[i,:]
        for j in range(x1.shape[0]):
            if round(d2.iloc[i,j],5) != round(d3.iloc[i,j],5):
                mismatch = True

    if(mismatch):
        print("Not Same")
    else:
        print("Both are same")

In [11]:
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
d2 = lda.fit_transform(X_train, y_train)
col_names = [ 'lda'+str(i) for i in range(0,9) ]
digit_lda_9 = pd.DataFrame(d2, columns = col_names)

Y = pd.DataFrame(np.array(y_train), columns=["label"])

d2 = pd.concat([digit_lda_9, Y], axis=1)
d2.head(10)

/Users/a5sharma/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/a5sharma/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


,lda0,lda1,lda2,lda3,lda4,lda5,lda6,lda7,lda8,label
0,-2.887029,-3.310528,-2.786562,0.058788,-3.018495,-0.674304,0.506955,-0.842984,-0.173764,0
1,-1.894420,-2.425312,-3.999679,-0.175208,-2.729570,-1.044016,1.235900,0.358109,-0.262772,0
2,-2.860135,-0.965014,-0.857678,-0.730400,-0.502527,0.296998,-1.284739,-1.513364,1.468125,0
3,-2.865351,-2.825316,-4.989777,0.231808,-0.884555,-1.196797,-0.355758,-0.332597,0.916868,0
4,-3.586286,-4.069599,-4.047539,-0.915669,-2.696322,-1.525039,-1.641412,-0.083102,-0.023521,0
5,-3.456195,-2.977401,-2.439679,-0.070588,0.010821,0.594510,-0.426037,-1.286878,1.460801,0
6,-2.362356,-1.329942,-3.998065,-0.273734,-1.020206,0.205699,1.455731,-1.140298,0.955281,0
7,-3.368090,-2.663745,-2.640072,-0.351479,-0.496984,0.606974,-0.583543,-0.090488,0.335246,0
8,-2.794333,-2.440993,-4.080267,0.533639,-2.163575,-1.261034,0.077082,-1.212496,0.325180,0
9,-3.801034,-4.642088,-3.061066,-0.319711,-3.782619,-1.189575,0.343004,-1.708295,0.630156,0


##### Build a Bayesian classifier on	D1 (single Gaussian per class)
Diagonal Covariance matrix (i.e.set non-diagonals to zero)<br>
Full Covariance matrix


In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.stats import multivariate_normal as mvn

class BayesianClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, covariance_type='full'):
        self.covariance_type = covariance_type
        self.gaussians = dict()
        self.priors = dict()
            
    def fit(self, X, y):
        labels = set(y)
        for c in labels:
            current_x = X[y==c]
            if self.covariance_type == 'full':
                cov = current_x.cov()
            elif self.covariance_type == 'diag':
                cov = current_x.cov()
                cov = np.diag(np.diag(cov))
            self.gaussians[c] = {
                'mean' : current_x.mean(axis=0),
                'cov': cov
            }
            self.priors[c] = float(len(y[y==c])) /len(y)
        return self
            
    def predict(self, X):
        N, D = X.shape
        K = len(self.gaussians)
        P = np.zeros((N,K))
        for c,g in self.gaussians.items():
            mean, cov = g['mean'], g['cov']
            P[:,c]=mvn.logpdf(X, mean=mean, cov=cov) + np.log(self.priors[c])
        return np.argmax(P, axis=1)

In [13]:
X_train = d1.iloc[:,0:9]
y_train = d1.label

model1 = BayesianClassifier(covariance_type='diag')
model1.fit(X_train, y_train)

y_pred = model1.predict(X_train)
correct = len(y_train[y_pred==y_train])/len(y_train)
incorrect = len(y_train[y_pred!=y_train])/len(y_train)
print('correct classified: ',correct)
print('Incorrect classified: ',incorrect)
print("acuuracy: {0}".format(correct))
metrics.confusion_matrix(y_train, y_pred)

correct classified:  0.7616503496503496
Incorrect classified:  0.23834965034965036
acuuracy: 0.7616503496503496


array([[2914,    0,   29,   84,    8,  366,  108,    7,   10,    4],
       [   0, 3761,  104,   21,    1,   52,   25,    7,   34,    6],
       [  56,   59, 2681,  109,  146,   49,  224,   61,  153,   15],
       [  35,   33,   97, 2638,   20,  343,   19,   41,  348,   76],
       [   2,   71,   20,    3, 2520,   25,   84,   18,   38,  704],
       [ 136,   91,   48,  377,  100, 2117,   57,   30,  118,  151],
       [  45,   80,  184,   39,   48,  162, 2962,    0,    7,    2],
       [  21,  188,   56,    2,  115,   59,    3, 3066,   69,  151],
       [  39,  117,   97,  299,   35,  195,   27,   17, 2513,  151],
       [  26,   61,   45,   39,  852,   81,    8,  250,  128, 2057]])

In [14]:
X_train = d1.iloc[:,0:9]
y_train = d1.label

model2 = BayesianClassifier(covariance_type='full')
model2.fit(X_train, y_train)

y_pred = model2.predict(X_train)
correct = len(y_train[y_pred==y_train])/len(y_train)
incorrect = len(y_train[y_pred!=y_train])/len(y_train)
print('correct classified: ',correct)
print('Incorrect classified: ',incorrect)
print("acuuracy: {0}".format(correct))
metrics.confusion_matrix(y_train, y_pred)

correct classified:  0.8747132867132867
Incorrect classified:  0.12528671328671329
acuuracy: 0.8747132867132867


array([[3336,    0,   13,   50,    4,   91,   21,    0,   12,    3],
       [   0, 3856,   50,   29,    9,    7,    5,    4,   35,   16],
       [  77,    3, 3126,   50,   99,   12,   79,   30,   61,   16],
       [  27,   13,   75, 3089,    9,  115,    6,   37,  209,   70],
       [  11,   11,   25,    3, 2907,    6,   49,   35,   19,  419],
       [  99,    3,   13,   81,   48, 2786,   47,   21,   69,   58],
       [  70,    6,   13,    1,   16,   52, 3342,    0,   25,    4],
       [   8,   28,   78,    1,   40,   14,    2, 3319,   50,  190],
       [  22,   57,   42,  206,   32,   94,   18,   14, 2899,  106],
       [  31,   19,    8,   58,  529,   25,   17,  147,  102, 2611]])

##### Build a Bayesian classifier on D2 (single Gaussian per class)
Diagonal Covariance<br>
Full covariance

In [15]:
X_train = d2.iloc[:,0:9]
y_train = d2.label

model3 = BayesianClassifier(covariance_type='diag')
model3.fit(X_train, y_train)

y_pred = model3.predict(X_train)
correct = len(y_train[y_pred==y_train])/len(y_train)
incorrect = len(y_train[y_pred!=y_train])/len(y_train)
print('correct classified: ',correct)
print('Incorrect classified: ',incorrect)
print("acuuracy: {0}".format(correct))
metrics.confusion_matrix(y_train, y_pred)

correct classified:  0.8842797202797202
Incorrect classified:  0.11572027972027972
acuuracy: 0.8842797202797202


array([[3330,    1,   24,   12,    5,   60,   45,    2,   46,    5],
       [   1, 3723,   56,   13,    4,   13,    5,    5,  186,    5],
       [  21,   22, 3090,  106,   40,   20,   90,   20,  133,   11],
       [   3,   26,  129, 3098,    4,  154,   16,   50,  119,   51],
       [   4,    8,   37,    4, 3135,   27,   23,    4,   50,  193],
       [  27,    4,   50,  109,   17, 2702,   67,   27,  164,   58],
       [  28,    3,   71,    3,   27,   78, 3273,    0,   46,    0],
       [  19,   38,   34,   33,   63,    7,    2, 3274,   25,  235],
       [  16,  104,   66,   89,   26,  170,   16,   11, 2933,   59],
       [  22,    7,   19,   45,  171,   17,    1,  155,   55, 3055]])

In [16]:
X_train = d2.iloc[:,0:9]
y_train = d2.label

model4 = BayesianClassifier(covariance_type='full')
model4.fit(X_train, y_train)

y_pred = model4.predict(X_train)
correct = len(y_train[y_pred==y_train])/len(y_train)
incorrect = len(y_train[y_pred!=y_train])/len(y_train)
print('correct classified: ',correct)
print('Incorrect classified: ',incorrect)
print("acuuracy: {0}".format(correct))
metrics.confusion_matrix(y_train, y_pred)

correct classified:  0.8969790209790209
Incorrect classified:  0.10302097902097902
acuuracy: 0.8969790209790209


array([[3395,    0,   26,    7,    4,   33,   35,    4,   22,    4],
       [   2, 3756,   58,   18,    3,    9,    1,   12,  148,    4],
       [  31,   13, 3188,   83,   35,   21,   63,   23,   89,    7],
       [  10,   13,  117, 3147,    4,  145,    9,   49,  114,   42],
       [   4,    2,   44,    2, 3127,   23,   41,   23,   44,  175],
       [  51,    5,   53,  109,   17, 2692,   58,   19,  176,   45],
       [  33,    2,   55,    2,    7,   76, 3335,    0,   19,    0],
       [   7,   11,   55,   37,   39,   13,    0, 3419,   41,  108],
       [  24,  100,   62,   90,   40,  149,   39,    8, 2929,   49],
       [  22,    2,   27,   52,  163,   25,    5,  136,   36, 3079]])

#### Compare the test accuracies of the four classifiers and comment.


In [17]:
col_names = [ 'pca'+str(i) for i in range(0,9) ]
X_trans_test = pd.DataFrame(pca.transform(X_test), columns = col_names)

y_test_pred = model1.predict(X_trans_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("acuuracy of model 1 for testing dataset: {0}".format(accuracy))
metrics.confusion_matrix(y_test, y_test_pred)

acuuracy of model 1 for testing dataset: 0.7585154639175258


array([[1990,    0,   21,   57,    5,  249,   62,    1,    5,    3],
       [   0, 2560,   65,   12,    2,   35,   19,   10,   25,    3],
       [  47,   45, 1814,   68,  108,   32,  130,   41,  108,   12],
       [  34,   24,   64, 1780,   12,  242,   13,   26,  234,   52],
       [   3,   50,   33,    1, 1633,   13,   57,   10,   38,  519],
       [  70,   62,   35,  240,   67, 1477,   35,   22,   82,  106],
       [  31,   67,  114,   20,   34,  111, 2003,    0,    5,    4],
       [  21,  124,   35,    1,   78,   34,    4, 2072,   43,  123],
       [  38,   67,   52,  233,   31,  126,   12,   11, 1675,  116],
       [  15,   40,   37,   23,  590,   49,    5,  156,   97, 1390]])

In [18]:
y_test_pred = model2.predict(X_trans_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("acuuracy of model 2 for testing dataset: {0}".format(accuracy))
metrics.confusion_matrix(y_test, y_test_pred)

acuuracy of model 2 for testing dataset: 0.8715876288659794


array([[2254,    0,    7,   28,    5,   65,   17,    1,   13,    3],
       [   0, 2626,   31,   18,    8,    3,    1,    3,   30,   11],
       [  61,    3, 2099,   32,   82,   13,   44,   18,   43,   10],
       [  31,   11,   51, 2072,    8,   76,    7,   27,  141,   57],
       [   7,    9,   20,    1, 1957,    6,   34,   18,   20,  285],
       [  58,    1,    9,   64,   36, 1907,   32,   11,   46,   32],
       [  52,    7,   12,    0,   17,   42, 2234,    0,   24,    1],
       [  11,   10,   44,    3,   23,   12,    1, 2265,   41,  125],
       [  21,   33,   31,  145,   23,   68,    8,   14, 1931,   87],
       [  18,   15,    4,   47,  334,   20,   13,   91,   69, 1791]])

In [19]:
col_names = [ 'lda'+str(i) for i in range(0,9) ]
X_trans_test = pd.DataFrame(lda.transform(X_test), columns = col_names)

y_test_pred = model3.predict(X_trans_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("acuuracy of model 3 for testing dataset: {0}".format(accuracy))
metrics.confusion_matrix(y_test, y_test_pred)

acuuracy of model 3 for testing dataset: 0.870020618556701


array([[2252,    0,   27,    4,    3,   38,   27,    2,   40,    0],
       [   0, 2525,   42,    6,    3,    9,    3,    6,  133,    4],
       [  22,   22, 2063,   72,   40,   10,   61,   22,   84,    9],
       [   3,   20,  108, 2041,    4,  119,    7,   56,   91,   32],
       [   3,   10,   32,    0, 2094,   20,   14,    4,   41,  139],
       [  22,    5,   24,   85,   11, 1788,   56,   27,  129,   49],
       [  34,    3,   56,    1,   21,   62, 2170,    0,   41,    1],
       [  25,   31,   30,   25,   58,    9,    0, 2170,    8,  179],
       [  16,   73,   32,   81,   19,  114,   16,    6, 1952,   52],
       [  19,    6,    7,   37,  116,   14,    1,  120,   39, 2043]])

In [20]:
y_test_pred = model4.predict(X_trans_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("acuuracy of model 4 for testing dataset: {0}".format(accuracy))
metrics.confusion_matrix(y_test, y_test_pred)

acuuracy of model 4 for testing dataset: 0.8845773195876289


array([[2285,    0,   34,    1,    2,   26,   19,    1,   23,    2],
       [   0, 2542,   49,    7,    5,    1,    2,    9,  112,    4],
       [  23,   11, 2167,   53,   30,   13,   25,   20,   55,    8],
       [  14,    7,  104, 2079,    4,  114,    1,   46,   85,   27],
       [   4,    4,   34,    1, 2107,   14,   21,   10,   37,  125],
       [  40,    4,   30,   79,   11, 1800,   47,   13,  131,   41],
       [  20,    2,   56,    1,   10,   78, 2202,    0,   20,    0],
       [  11,    7,   52,   23,   34,   13,    0, 2273,   32,   90],
       [  14,   60,   39,   76,   24,  126,   21,    5, 1945,   51],
       [  16,    1,   14,   39,  112,   28,    2,  100,   39, 2051]])

###### Model 1: PCA transformation + BayesianClassifier with covariance_type = 'diag' (d1 dataset)

acuuracy of model 1 for training dataset: 0.7616503496503496<br>
acuuracy of model 1 for testing dataset: 0.7585154639175258

###### Model 2: PCA transformation + BayesianClassifier with covariance_type = 'full' (d1 dataset)

acuuracy of model 2 for training dataset: 0.8747132867132867<br>
acuuracy of model 2 for testing dataset: 0.8715876288659794

###### Model 3: LDA transformation + BayesianClassifier with covariance_type = 'diag' (d2 dataset)

acuuracy of model 3 for training dataset: 0.8842797202797202<br>
acuuracy of model 3 for testing dataset: 0.870020618556701

###### Model 4: LDA transformation + BayesianClassifier with covariance_type = 'full' (d2 dataset)

acuuracy of model 4 for training dataset: 0.8969790209790209<br>
acuuracy of model 4 for testing dataset: 0.8845773195876289

We can see that model 4 accuracy for training and testing dataset is very high compare than other model. Hence, model4 is a good model.